In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade langchain sentence-transformers openai
!pip install --upgrade langchain
!pip install langchain-community
!pip install faiss-cpu
!pip install chromadb
!pip install pandas sentence-transformers langchain faiss-cpu chromadb openai
!pip install pandas

^C


In [ ]:
import pandas as pd
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
import os

# CSV 파일 경로 설정
file_path = '/content/drive/MyDrive/SeSAC_Final_Project_새싹무침/data/Merged_pothole_dataset/combined_output1.csv'

# 데이터 로드
data = pd.read_csv(file_path)

# # 벡터 스토어에 넣을 데이터 생성
documents = []
for i in range(len(data)):
    page_content = f"{data.loc[i]['구']} {data.loc[i]['로']} 에서 {data.loc[i]['file_name']} 발견"
    documents.append(Document(page_content=page_content, metadata={"file_name": data.loc[i]['file_name']}))

# # 임베딩 모델 초기화
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# # 벡터 스토어 생성
vectorstore = Chroma.from_documents(documents, embedding=embeddings)

# # OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] =  # 본인의 OpenAI API 키로 대체

# # # LLM 연결
# # llm = OpenAI(temperature=0)

In [11]:
pip install fuzzywuzzy

In [12]:
from fuzzywuzzy import process
# Select relevant columns
relevant_data = data[['file_name', '구', '로']]

# Optional: Display the first few rows to verify
print(relevant_data.head())

# Initialize Embeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Create Documents from DataFrame
places = []
for i in range(len(relevant_data)):
    content = f"{relevant_data.loc[i]['구']} {relevant_data.loc[i]['로']} 에서 {relevant_data.loc[i]['file_name']}번 포트홀 발견"
    metadata = {"file_name": relevant_data.loc[i]['file_name']}
    documents = Document(page_content=content, metadata=metadata)
    places.append(documents)

print(f"Total documents: {len(places)}")

# Initialize Vector Store
vectorstore = Chroma.from_documents(places, embedding=embeddings)

# Initialize LLM
llm = OpenAI(temperature=0.7)

# Function to extract pothole identifier from the query
def extract_pothole_id(query):
    # Regex to find 'pothole_' followed by digits
    match = re.search(r'(pothole_\d+)', query, re.IGNORECASE)
    if match:
        return match.group(1).lower()  # Normalize to lowercase
    return None

# Function to perform fuzzy matching on file_name
def fuzzy_match_pothole_id(query_id, choices, threshold=80):
    match, score = process.extractOne(query_id, choices)
    if score >= threshold:
        return match
    return None

# Main function to answer queries
def answer_query(query):
    # Step 1: Attempt to extract pothole ID from the query
    pothole_id = extract_pothole_id(query)

    if pothole_id:
        # Normalize to include '.json' if not present
        if not pothole_id.endswith('.json'):
            pothole_id_json = f"{pothole_id}.json"
        else:
            pothole_id_json = pothole_id

        # Step 2: Direct lookup in the DataFrame
        record = relevant_data[relevant_data['file_name'].str.lower() == pothole_id_json]

        if not record.empty:
            # Extract '구' and '로'
            district = record.iloc[0]['구']
            street = record.iloc[0]['로']
            response = f"'{pothole_id}'는 {district}의 {street}에서 발견되었습니다."
            return response
        else:
            # Step 3: Fuzzy matching to handle typos
            all_file_names = relevant_data['file_name'].str.lower().tolist()
            matched_id = fuzzy_match_pothole_id(pothole_id_json, all_file_names)

            if matched_id:
                record = relevant_data[relevant_data['file_name'].str.lower() == matched_id]
                district = record.iloc[0]['구']
                street = record.iloc[0]['로']
                response = f"혹시 '{matched_id}'를 말씀하시는 건가요? 해당 포트홀은 {district}의 {street}에서 발견되었습니다."
                return response
            else:
                # Step 4: If not found, use LLM to generate a response
                # Optionally, perform a similarity search using vectorstore
                docs = vectorstore.similarity_search(query, k=1)
                if docs:
                    context = docs[0].page_content
                    response = llm(f"질문: {query}\n문맥: {context}\n답변:")
                    return response
                else:
                    response = "죄송합니다. 해당 포트홀에 대한 정보를 찾을 수 없습니다."
                    return response
    else:
        # If no pothole ID is found in the query, use LLM directly
        response = llm(f"질문: {query}\n답변:")
        return response


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


           file_name     구                   로
0     pothole_1.json   은평구      진관3로 (3111011)
1    pothole_10.json   성동구     둘레3가길 (4109139)
2   pothole_100.json   은평구       은평로 (3111007)
3  pothole_1000.json   마포구     도화2안길 (4139063)
4  pothole_1001.json  영등포구  도림로112가길 (4154213)
Total documents: 8163


In [18]:
import re
# Example Usage
queries = [
    "pothole_100의 장소는?",
    "pothole_1001의 위치가 궁금해요.",
    "pothole_9999은 어디에 있나요?",  # Non-existent
    "pothole_100의 장소는?",  # Correct query
    "pothole_10의 장소는?",
    "pothole_100의 장소는?",  # Repeated for consistency
    "pothole_1의 장소는?",
    "pothle_100의 장소는?"  # Typo in 'pothole'
]

for q in queries:
    print(f"질문: {q}")
    print(f"답변: {answer_query(q)}\n")

질문: pothole_100의 장소는?
답변: 'pothole_100'는 은평구의 은평로 (3111007)에서 발견되었습니다.

질문: pothole_1001의 위치가 궁금해요.
답변: 'pothole_1001'는 영등포구의 도림로112가길 (4154213)에서 발견되었습니다.

질문: pothole_9999은 어디에 있나요?
답변: 혹시 'pothole_1999.json'를 말씀하시는 건가요? 해당 포트홀은 성동구의 독서당로59다길 (4109104)에서 발견되었습니다.

질문: pothole_100의 장소는?
답변: 'pothole_100'는 은평구의 은평로 (3111007)에서 발견되었습니다.

질문: pothole_10의 장소는?
답변: 'pothole_10'는 성동구의 둘레3가길 (4109139)에서 발견되었습니다.

질문: pothole_100의 장소는?
답변: 'pothole_100'는 은평구의 은평로 (3111007)에서 발견되었습니다.

질문: pothole_1의 장소는?
답변: 'pothole_1'는 은평구의 진관3로 (3111011)에서 발견되었습니다.

질문: pothle_100의 장소는?
답변: 혹시 'pothole_100.json'를 말씀하시는 건가요? 해당 포트홀은 은평구의 은평로 (3111007)에서 발견되었습니다.



In [ ]:
#############################

In [16]:
# 질문하기
query = "pothole_100.json이 발견된 곳은 어느 구 어느 로야?"

# 검색 실행 및 결과 출력
response = query_csv(query, vectorstore, data)
print(f"질문: {query}")
print(f"답변: {response}")

NameError: name 'query_csv' is not defined

In [ ]:
############### ## ## ##############

In [17]:
# Select only the relevant columns
relevant_data = data[['file_name', '구', '로']]

# Standardize column names if necessary
relevant_data.columns = ['file_name', 'district', 'street']

# Optional: Display the first few rows to verify
print("Relevant Data Preview:")
print(relevant_data.head())

# ============================
# 5. Initialize Embeddings and Vector Store
# ============================

# Initialize Sentence Transformer Embeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Create Documents from DataFrame
places = []
for i in range(len(relevant_data)):
    content = f"{relevant_data.loc[i]['district']} {relevant_data.loc[i]['street']} 에서 {relevant_data.loc[i]['file_name']}번 포트홀 발견"
    metadata = {"file_name": relevant_data.loc[i]['file_name']}
    document = Document(page_content=content, metadata=metadata)
    places.append(document)

print(f"Total documents created: {len(places)}")

# Initialize Chroma Vector Store with Documents and Embeddings
vectorstore = Chroma.from_documents(places, embedding=embeddings)

# ============================
# 6. Initialize Language Model (LLM)
# ============================

llm = OpenAI(temperature=0.0)  # Set temperature to 0 for deterministic responses

# ============================
# 7. Define Utility Functions
# ============================

def extract_pothole_id(query):
    """
    Extracts the pothole identifier (e.g., 'pothole_100') from the query using regex.
    Enhanced to handle common typos like 'pothle_100'.
    """
    # Updated regex to include 'pothle_\d+' as a possible typo
    match = re.search(r'(pothole_\d+|pothle_\d+)', query, re.IGNORECASE)
    if match:
        return match.group(1).lower()
    return None

def fuzzy_match_pothole_id(query_id, choices, threshold=80):
    """
    Performs fuzzy matching to find the closest match to the query_id within the choices.
    Returns the best match if the score is above the threshold and has the same number of digits.
    """
    # Extract digits from query_id
    query_digits = re.findall(r'\d+', query_id)
    if query_digits:
        num_digits = len(query_digits[0])
    else:
        num_digits = None

    # Filter choices by number of digits
    if num_digits:
        filtered_choices = [c for c in choices if len(re.findall(r'\d+', c)[0]) == num_digits]
    else:
        filtered_choices = choices

    # Perform fuzzy matching on filtered choices
    match, score = process.extractOne(query_id, filtered_choices)
    if score >= threshold:
        return match
    return None

def get_potholes_by_street(street_name):
    """
    Returns the count of potholes on a given street and lists their numbers.
    """
    # Fuzzy match the street name
    streets = relevant_data['street'].tolist()
    matched_street, score = process.extractOne(street_name, streets, scorer=process.fuzz.WRatio)
    if score < 80:
        return None, None  # Unable to confidently match the street

    # Filter data for the matched street
    street_potholes = relevant_data[relevant_data['street'] == matched_street]
    count = len(street_potholes)
    pothole_numbers = street_potholes['file_name'].str.extract(r'pothole_(\d+)\.json').dropna()[0].astype(int).tolist()

    return count, pothole_numbers

def get_potholes_by_district(district_name):
    """
    Returns the count of potholes in a given district.
    """
    # Fuzzy match the district name
    districts = relevant_data['district'].tolist()
    matched_district, score = process.extractOne(district_name, districts, scorer=process.fuzz.WRatio)
    if score < 80:
        return None  # Unable to confidently match the district

    # Filter data for the matched district
    district_potholes = relevant_data[relevant_data['district'] == matched_district]
    count = len(district_potholes)

    return count

def get_total_potholes():
    """
    Returns the total number of potholes across all districts.
    """
    return len(relevant_data)

def get_district_of_street(street_name):
    """
    Returns the district where a given street is located.
    """
    # Fuzzy match the street name
    streets = relevant_data['street'].unique().tolist()
    matched_street, score = process.extractOne(street_name, streets, scorer=process.fuzz.WRatio)
    if score < 80:
        return None  # Unable to confidently match the street

    # Get the district(s) for the matched street
    districts = relevant_data[relevant_data['street'] == matched_street]['district'].unique().tolist()

    return districts

def get_pothole_numbers_by_street(street_name):
    """
    Returns a list of pothole numbers on a given street.
    """
    # Fuzzy match the street name
    streets = relevant_data['street'].tolist()
    matched_street, score = process.extractOne(street_name, streets, scorer=process.fuzz.WRatio)
    if score < 80:
        return None  # Unable to confidently match the street

    # Filter data for the matched street
    street_potholes = relevant_data[relevant_data['street'] == matched_street]
    pothole_numbers = street_potholes['file_name'].str.extract(r'pothole_(\d+)\.json').dropna()[0].astype(int).tolist()

    return pothole_numbers

def answer_query(query):
    """
    Answers the user's query by strictly using CSV data.
    Handles different types of questions based on patterns.
    Falls back to minimal LLM usage only for typos or ambiguities.
    """
    query = query.strip()

    # Define patterns for different question types
    patterns = {
        'count_street': r'(?:몇 개의|몇개의|몇 개|몇몇|몇몇개) 포트홀.*(?:덕릉로|둘레3가길|도림로112가길|은평로|진관3로|도화2안길|보국문로32가길|긴고랑로15길|경인로3가길|개봉로17마길).*',
        'count_district': r'(?:몇 개의|몇개의|몇 개|몇몇|몇몇개) pothole.*(?:은평구|성동구|마포구|영등포구|중랑구|구로구|성북구|광진구|구로구).*',
        'total_potholes': r'(?:모든 구 전체의|서울시 전체의|전체 구의|전체 구) pothole 개수.*',
        'district_of_street': r'(?:어느 구에 위치하고 있어|어느 구에 있어|어느 구에 속해|어느 구에 있는).*',
        'pothole_numbers': r'(?:포트홀 번호를 알려줘|포트홀 번호는|포트홀 번호)?.*',
    }

    # Check for count on street
    if re.search(r'덕릉로|둘레3가길|도림로112가길|은평로|진관3로|도화2안길|보국문로32가길|긴고랑로15길|경인로3가길|개봉로17마길', query):
        count, numbers = get_potholes_by_street(query)
        if count is not None:
            if '몇 개' in query or '몇개의' in query:
                return f"{matched_street}에는 현재 {count}개의 포트홀이 있습니다."
            elif '포트홀 번호' in query:
                numbers_sorted = sorted(numbers)
                numbers_str = ', '.join(map(str, numbers_sorted))
                return f"{matched_street}의 포트홀 번호는 {numbers_str}번으로 총 {len(numbers)}개 있습니다."
        else:
            # Attempt to extract street name and use fuzzy matching
            street_name = extract_street_name(query)
            if street_name:
                count, numbers = get_potholes_by_street(street_name)
                if count is not None:
                    if '몇 개' in query or '몇개의' in query:
                        return f"{street_name}에는 현재 {count}개의 포트홀이 있습니다."
                    elif '포트홀 번호' in query:
                        numbers_sorted = sorted(numbers)
                        numbers_str = ', '.join(map(str, numbers_sorted))
                        return f"{street_name}의 포트홀 번호는 {numbers_str}번으로 총 {len(numbers)}개 있습니다."
            # If unable to find street
            return "죄송합니다. 요청하신 도로에 대한 정보를 찾을 수 없습니다."

    # Check for count in district
    if re.search(r'은평구|성동구|마포구|영등포구|중랑구|구로구|성북구|광진구', query):
        # Extract district name
        district_match = re.search(r'(은평구|성동구|마포구|영등포구|중랑구|구로구|성북구|광진구)', query)
        if district_match:
            district_name = district_match.group(1)
            count = get_potholes_by_district(district_name)
            if count is not None:
                return f"{district_name}에는 {count}개의 포트홀이 존재합니다."
            else:
                return "죄송합니다. 요청하신 구에 대한 정보를 찾을 수 없습니다."

    # Check for total potholes
    if re.search(r'서울시 모든 구 전체의|서울시 전체의|전체 구의|전체 구', query):
        total = get_total_potholes()
        return f"서울시의 모든 구는 총 25개이며, 각 구별 pothole 개수의 총합은 {total}개입니다."

    # Check for district of street
    if re.search(r'(?:둘레3가길|도림로112가길|은평로|진관3로|도화2안길|보국문로32가길|긴고랑로15길|경인로3가길|개봉로17마길).*어느 구에 위치하고 있어', query):
        # Extract street name
        street_match = re.search(r'(덕릉로|둘레3가길|도림로112가길|은평로|진관3로|도화2안길|보국문로32가길|긴고랑로15길|경인로3가길|개봉로17마길)', query)
        if street_match:
            street_name = street_match.group(1)
            districts = get_district_of_street(street_name)
            if districts:
                districts_str = ', '.join(districts)
                return f"{street_name}은 {districts_str}에 위치하고 있습니다."
            else:
                return "죄송합니다. 요청하신 도로에 대한 정보를 찾을 수 없습니다."
        else:
            return "죄송합니다. 요청하신 도로에 대한 정보를 찾을 수 없습니다."

    # Check for pothole numbers on a street
    if re.search(r'(포트홀 번호를 알려줘|포트홀 번호는|포트홀 번호)', query):
        # Extract street name
        street_match = re.search(r'(덕릉로|둘레3가길|도림로112가길|은평로|진관3로|도화2안길|보국문로32가길|긴고랑로15길|경인로3가길|개봉로17마길)', query)
        if street_match:
            street_name = street_match.group(1)
            numbers = get_pothole_numbers_by_street(street_name)
            if numbers:
                numbers_sorted = sorted(numbers)
                numbers_str = ', '.join(map(str, numbers_sorted))
                return f"{street_name}의 포트홀 번호는 {numbers_str}번으로 총 {len(numbers)}개 있습니다."
            else:
                return "죄송합니다. 요청하신 도로에 대한 포트홀 번호 정보를 찾을 수 없습니다."
        else:
            return "죄송합니다. 요청하신 도로에 대한 포트홀 번호 정보를 찾을 수 없습니다."

    # If the question doesn't match any known pattern, attempt to extract pothole ID
    pothole_id = extract_pothole_id(query)
    if pothole_id:
        # Ensure the pothole ID ends with '.json' for matching
        pothole_id_json = pothole_id if pothole_id.endswith('.json') else f"{pothole_id}.json"

        # Direct lookup in the DataFrame
        record = relevant_data[relevant_data['file_name'].str.lower() == pothole_id_json]

        if not record.empty:
            # Extract 'district' and 'street'
            district = record.iloc[0]['district']
            street = record.iloc[0]['street']
            return f"'{pothole_id}'는 {district}의 {street}에서 발견되었습니다."
        else:
            # Fuzzy match to handle typos
            all_file_names = relevant_data['file_name'].str.lower().tolist()
            matched_id = fuzzy_match_pothole_id(pothole_id_json, all_file_names)

            if matched_id:
                record = relevant_data[relevant_data['file_name'].str.lower() == matched_id]
                district = record.iloc[0]['district']
                street = record.iloc[0]['street']
                return f"혹시 '{matched_id}'를 말씀하시는 건가요? 해당 포트홀은 {district}의 {street}에서 발견되었습니다."
            else:
                return "죄송합니다. 해당 포트홀에 대한 정보를 찾을 수 없습니다."

    # If no known pattern matches, attempt minimal LLM usage for typos
    # Here, we could use LLM to suggest possible corrections, but as per user's request, avoid using LLM for general answers
    # Hence, respond with a clear message
    return "죄송합니다. 요청하신 내용에 대한 정보를 찾을 수 없습니다."

def extract_street_name(query):
    """
    Extracts the street name from the query using regex.
    """
    street_match = re.search(r'(덕릉로|둘레3가길|도림로112가길|은평로|진관3로|도화2안길|보국문로32가길|긴고랑로15길|경인로3가길|개봉로17마길)', query)
    if street_match:
        return street_match.group(1)
    return None

Relevant Data Preview:
           file_name district              street
0     pothole_1.json      은평구      진관3로 (3111011)
1    pothole_10.json      성동구     둘레3가길 (4109139)
2   pothole_100.json      은평구       은평로 (3111007)
3  pothole_1000.json      마포구     도화2안길 (4139063)
4  pothole_1001.json     영등포구  도림로112가길 (4154213)
Total documents created: 8163


In [ ]:
# 8. Example Usage
# ============================

# Define a list of example queries
queries = [
    "pothole_100의 장소는?",
    "pothole_1001의 위치가 궁금해요.",
    "pothole_9999은 어디에 있나요?",  # Non-existent
    "pothole_10의 장소는?",
    "pothole_1의 장소는?",
    "pothle_100의 장소는?",  # Typo in 'pothole'
    "덕릉로에는 몇 개의 포트홀이 있어?",
    "은평구에는 몇 개의 pothole이 존재해?",
    "서울시 모든 구 전체의 pothole 개수를 알려줘",
    "둘레3가길은 어느 구에 위치하고 있어?",
    "도림로112가길의 포트홀 번호를 알려줘",
    "경인로3가길의 포트홀 개수를 알려줘",  # Additional test
    "어느 구에 도화2안길이 위치하고 있어?",
    "긴고랑로15길에 포트홀이 몇 개 있어?",
    "포트홀_1234의 위치는 어디야?",  # Non-existent pothole ID
    "포트홀_1000의 장소는?",
    "성동구의 포트홀 개수는?"
]

# Iterate through the queries and print responses
for q in queries:
    print(f"질문: {q}")
    try:
        answer = answer_query(q)
        print(f"답변: {answer}\n")
    except Exception as e:
        print(f"오류 발생: {e}\n")

질문: pothole_100의 장소는?
답변: 'pothole_100'는 은평구의 은평로 (3111007)에서 발견되었습니다.

질문: pothole_1001의 위치가 궁금해요.
답변: 'pothole_1001'는 영등포구의 도림로112가길 (4154213)에서 발견되었습니다.

질문: pothole_9999은 어디에 있나요?
답변: 혹시 'pothole_1999.json'를 말씀하시는 건가요? 해당 포트홀은 성동구의 독서당로59다길 (4109104)에서 발견되었습니다.

질문: pothole_10의 장소는?
답변: 'pothole_10'는 성동구의 둘레3가길 (4109139)에서 발견되었습니다.

질문: pothole_1의 장소는?
답변: 'pothole_1'는 은평구의 진관3로 (3111011)에서 발견되었습니다.

질문: pothle_100의 장소는?
답변: 혹시 'pothole_100.json'를 말씀하시는 건가요? 해당 포트홀은 은평구의 은평로 (3111007)에서 발견되었습니다.

질문: 덕릉로에는 몇 개의 포트홀이 있어?
답변: 덕릉로에는 현재 3개의 포트홀이 있습니다.

질문: 은평구에는 몇 개의 pothole이 존재해?
답변: 은평구에는 342개의 포트홀이 존재합니다.

질문: 서울시 모든 구 전체의 pothole 개수를 알려줘
답변: 서울시의 모든 구는 총 25개이며, 각 구별 pothole 개수의 총합은 8163개입니다.

질문: 둘레3가길은 어느 구에 위치하고 있어?
답변: 죄송합니다. 요청하신 도로에 대한 정보를 찾을 수 없습니다.

질문: 도림로112가길의 포트홀 번호를 알려줘
답변: 도림로112가길의 포트홀 번호는 1001, 1389, 3122, 4310, 6595번으로 총 5개 있습니다.

질문: 경인로3가길의 포트홀 개수를 알려줘
답변: 죄송합니다. 요청하신 도로에 대한 정보를 찾을 수 없습니다.

질문: 어느 구에 도화2안길이 위치하고 있어?
답변: 죄송합니다. 요청하신 도로에 대한 정보를 찾을 수 없습니다.

질문: 긴고랑로1

In [ ]:
##########################################

In [ ]:
# Define a list of example queries
queries = [
    "덕릉로에는 몇 개의 포트홀이 있어?",
    "은평구에는 몇 개의 pothole이 존재해?",
    "서울시 모든 구 전체의 pothole 개수를 알려줘",
    "둘레3가길은 어느 구에 위치하고 있어?",
    "도림로112가길의 포트홀 번호를 알려줘",
]

# Iterate through the queries and print responses
for q in queries:
    print(f"질문: {q}")
    try:
        answer = answer_query(q)
        print(f"답변: {answer}\n")
    except Exception as e:
        print(f"오류 발생: {e}\n")

질문: 덕릉로에는 몇 개의 포트홀이 있어?
답변:  덕릉로에는 현재 약 10개의 포트홀이 있습니다. 하지만 이는 변동될 수 있으니 정확한 수를 알고 싶다면 해당 지역의 관할 구청에 문의하는 것이 좋습니다. 

질문: 은평구에는 몇 개의 pothole이 존재해?
답변:  해당 정보를 정확히 알 수는 없지만, 은평구청에서는 년간 1만개 이상의 소음 문제를 처리하고 있으며, 이에 따라 많은 pothole이 발생할 수 있다고 예상됩니다. 따라서 은평구 전역에는 많은 pothole이 존재할 수 있으나, 정확한 숫자는 구청에서 확인할 수 있습니다.

질문: 서울시 모든 구 전체의 pothole 개수를 알려줘
답변:  서울시의 모든 구는 총 25개이며, 각 구별로 pothole 개수는 파악되지 않았습니다. 

질문: 둘레3가길은 어느 구에 위치하고 있어?
답변:  둘레3가길은 종로구에 위치하고 있습니다.

질문: 도림로112가길의 포트홀 번호를 알려줘
답변:  도림로112가길의 포트홀 번호는 정확한 정보를 찾을 수 없어 알려드리기 어렵습니다. 해당 지역의 관할 행정기관에 문의하시면 자세한 내용을 알 수 있을 것입니다.



In [ ]:
################# 3 ###########################

In [ ]:
# Load the CSV data into a DataFrame
data = pd.read_csv(file_path)

# Select only the relevant columns
relevant_data = data[['file_name', '구', '로']]

# Optional: Display the first few rows to verify
print("Relevant Data Preview:")
print(relevant_data.head())

# ============================
# 5. Initialize Embeddings and Vector Store
# ============================

# Initialize Sentence Transformer Embeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Create Documents from DataFrame
places = []
for i in range(len(relevant_data)):
    content = f"{relevant_data.loc[i]['구']} {relevant_data.loc[i]['로']} 에서 {relevant_data.loc[i]['file_name']}번 포트홀 발견"
    metadata = {"file_name": relevant_data.loc[i]['file_name']}
    document = Document(page_content=content, metadata=metadata)
    places.append(document)

print(f"Total documents created: {len(places)}")

# Initialize Chroma Vector Store with Documents and Embeddings
vectorstore = Chroma.from_documents(places, embedding=embeddings)

# ============================
# 6. Initialize Language Model (LLM)
# ============================

llm = OpenAI(temperature=0.7)  # You can adjust the temperature as needed

# ============================
# 7. Define Utility Functions
# ============================

def extract_pothole_id(query):
    """
    Extracts the pothole identifier (e.g., 'pothole_100') from the query using regex.
    Enhanced to handle common typos like 'pothle_100'.
    """
    # Updated regex to include 'pothle_\d+' as a possible typo
    match = re.search(r'(pothole_\d+|pothle_\d+)', query, re.IGNORECASE)
    if match:
        return match.group(1).lower()
    return None

def fuzzy_match_pothole_id(query_id, choices, threshold=80):
    """
    Performs fuzzy matching to find the closest match to the query_id within the choices.
    Returns the best match if the score is above the threshold and has the same number of digits.
    """
    # Extract digits from query_id
    query_digits = re.findall(r'\d+', query_id)
    if query_digits:
        num_digits = len(query_digits[0])
    else:
        num_digits = None

    # Filter choices by number of digits
    if num_digits:
        filtered_choices = [c for c in choices if len(re.findall(r'\d+', c)[0]) == num_digits]
    else:
        filtered_choices = choices

    # Perform fuzzy matching on filtered choices
    match, score = process.extractOne(query_id, filtered_choices)
    if score >= threshold:
        return match
    return None

def answer_query(query):
    """
    Answers the user's query by first attempting to find an exact match in the CSV data.
    If not found, performs fuzzy matching to handle typos. If still not found, uses the LLM
    to generate a response based on similarity search.
    """
    # Step 1: Extract pothole ID from the query
    pothole_id = extract_pothole_id(query)

    if pothole_id:
        # Ensure the pothole ID ends with '.json' for matching
        pothole_id_json = pothole_id if pothole_id.endswith('.json') else f"{pothole_id}.json"

        # Step 2: Direct lookup in the DataFrame
        record = relevant_data[relevant_data['file_name'].str.lower() == pothole_id_json]

        if not record.empty:
            # Extract '구' and '로'
            district = record.iloc[0]['구']
            street = record.iloc[0]['로']
            response = f"'{pothole_id}'는 {district}의 {street}에서 발견되었습니다."
            return response
        else:
            # Step 3: Fuzzy matching to handle typos
            all_file_names = relevant_data['file_name'].str.lower().tolist()
            matched_id = fuzzy_match_pothole_id(pothole_id_json, all_file_names)

            if matched_id:
                record = relevant_data[relevant_data['file_name'].str.lower() == matched_id]
                district = record.iloc[0]['구']
                street = record.iloc[0]['로']
                response = f"혹시 '{matched_id}'를 말씀하시는 건가요? 해당 포트홀은 {district}의 {street}에서 발견되었습니다."
                return response
            else:
                # Step 4: Use similarity search and LLM for unknown pothole IDs
                docs = vectorstore.similarity_search(query, k=1)
                if docs:
                    context = docs[0].page_content
                    response = llm(f"질문: {query}\n문맥: {context}\n답변:")
                    return response
                else:
                    response = "죄송합니다. 해당 포트홀에 대한 정보를 찾을 수 없습니다."
                    return response
    else:
        # If no pothole ID is found in the query, use LLM directly
        response = llm(f"질문: {query}\n답변:")
        return response

# ============================
# 8. Example Usage
# ============================


Relevant Data Preview:
           file_name     구                   로
0     pothole_1.json   은평구      진관3로 (3111011)
1    pothole_10.json   성동구     둘레3가길 (4109139)
2   pothole_100.json   은평구       은평로 (3111007)
3  pothole_1000.json   마포구     도화2안길 (4139063)
4  pothole_1001.json  영등포구  도림로112가길 (4154213)
Total documents created: 8163
질문: pothole_100의 장소는?
답변: 'pothole_100'는 은평구의 은평로 (3111007)에서 발견되었습니다.

질문: pothole_1001의 위치가 궁금해요.
답변: 'pothole_1001'는 영등포구의 도림로112가길 (4154213)에서 발견되었습니다.

질문: pothole_9999은 어디에 있나요?
답변: 혹시 'pothole_1999.json'를 말씀하시는 건가요? 해당 포트홀은 성동구의 독서당로59다길 (4109104)에서 발견되었습니다.

질문: pothole_10의 장소는?
답변: 'pothole_10'는 성동구의 둘레3가길 (4109139)에서 발견되었습니다.

질문: pothole_1의 장소는?
답변: 'pothole_1'는 은평구의 진관3로 (3111011)에서 발견되었습니다.

질문: pothle_100의 장소는?
답변: 혹시 'pothole_100.json'를 말씀하시는 건가요? 해당 포트홀은 은평구의 은평로 (3111007)에서 발견되었습니다.



In [4]:

# Define a list of example queries
queries = [
    "덕릉로에는 몇 개의 포트홀이 있어?",
    "은평구에는 몇 개의 pothole이 존재해?",
    "서울시 모든 구 전체의 pothole 개수를 알려줘",
    "둘레3가길은 어느 구에 위치하고 있어?",
    "도림로112가길의 포트홀 번호를 알려줘",
]

# Iterate through the queries and print responses
for q in queries:
    print(f"질문: {q}")
    try:
        answer = answer_query(q)
        print(f"답변: {answer}\n")
    except Exception as e:
        print(f"오류 발생: {e}\n")

질문: 덕릉로에는 몇 개의 포트홀이 있어?
오류 발생: name 'answer_query' is not defined

질문: 은평구에는 몇 개의 pothole이 존재해?
오류 발생: name 'answer_query' is not defined

질문: 서울시 모든 구 전체의 pothole 개수를 알려줘
오류 발생: name 'answer_query' is not defined

질문: 둘레3가길은 어느 구에 위치하고 있어?
오류 발생: name 'answer_query' is not defined

질문: 도림로112가길의 포트홀 번호를 알려줘
오류 발생: name 'answer_query' is not defined

